# Ejercicio 4

In [29]:
import os.path

data_base_path = "./data"

In [30]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmread

X_train = csr_matrix(mmread(os.path.join(data_base_path, 'train.x.mm')))
y_train = np.loadtxt(os.path.join(data_base_path, 'train.y.dat'))

X_test = csr_matrix(mmread(os.path.join(data_base_path, 'test.x.mm')))
y_test = np.loadtxt(os.path.join(data_base_path, 'test.y.dat'))

In [31]:
X_train

<1147x145256 sparse matrix of type '<type 'numpy.float64'>'
	with 2764110 stored elements in Compressed Sparse Row format>

In [32]:
#from sklearn.preprocessing import StandardScaler

#x_scaler = StandardScaler(with_mean=False)

#X_train = x_scaler.fit_transform(X_train, y_train)
#X_test = x_scaler.transform(X_test, y_test)

In [33]:
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=30000)

X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

In [34]:
import sklearn.linear_model as lm

model = lm.LinearRegression(fit_intercept = False)
model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

R2=0.598126


In [35]:
model = lm.Lasso(1000., fit_intercept = False, normalize=True)
model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

R2=0.533190
